In [56]:
# | echo: false
# ---
# title: "matplotlib example"
# format:
#   html:
#     code-fold: true
# ---

In [57]:
#| echo: false
# Supress warnings.
import warnings
warnings.filterwarnings('ignore')
warnings.warn('DelftStack')
warnings.warn('Do not show this message')

In [84]:
# Import necessary libraries
# Importing libraries 
#| echo: false
import numpy as np 
import pandas as pd
import janitor
import sklearn 
from sklearn.impute import KNNImputer
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, precision_score, accuracy_score, recall_score, f1_score
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from copy import deepcopy
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.decomposition import PCA

In [59]:
#| echo: false
data = pd.read_csv('../data/mushrooms.csv')

## Overview of the dataset

description, dictionary

Mushroom anatomy picture

Objective

## Data Munging

1) Clean column names automatically by replacing each - with _ using  `pyjanitor`

In [60]:
#| echo: true
print("Column names before cleaning:", '\n', data.columns.tolist(), '\n\n')
data = data.clean_names()
print("Column names after cleaning:", '\n', data.columns.tolist()) 

Column names before cleaning: 
 ['class', 'cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor', 'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color', 'stalk-shape', 'stalk-root', 'stalk-surface-above-ring', 'stalk-surface-below-ring', 'stalk-color-above-ring', 'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number', 'ring-type', 'spore-print-color', 'population', 'habitat'] 


Column names after cleaning: 
 ['class', 'cap_shape', 'cap_surface', 'cap_color', 'bruises', 'odor', 'gill_attachment', 'gill_spacing', 'gill_size', 'gill_color', 'stalk_shape', 'stalk_root', 'stalk_surface_above_ring', 'stalk_surface_below_ring', 'stalk_color_above_ring', 'stalk_color_below_ring', 'veil_type', 'veil_color', 'ring_number', 'ring_type', 'spore_print_color', 'population', 'habitat']


2) See what different values each column contains

From here, we can see that the `veil_type` has one single value and therefore is redundant and not informative so we can proceed with dropping it.
All the mushrooms in our dataset have partial veils, so the column `veil_type` is not informative. 

In [61]:
#| echo: false
data.columns.tolist()
for col in data.columns.tolist(): 
    print(col,':  ',data[col].unique())

class :   ['p' 'e']
cap_shape :   ['x' 'b' 's' 'f' 'k' 'c']
cap_surface :   ['s' 'y' 'f' 'g']
cap_color :   ['n' 'y' 'w' 'g' 'e' 'p' 'b' 'u' 'c' 'r']
bruises :   ['t' 'f']
odor :   ['p' 'a' 'l' 'n' 'f' 'c' 'y' 's' 'm']
gill_attachment :   ['f' 'a']
gill_spacing :   ['c' 'w']
gill_size :   ['n' 'b']
gill_color :   ['k' 'n' 'g' 'p' 'w' 'h' 'u' 'e' 'b' 'r' 'y' 'o']
stalk_shape :   ['e' 't']
stalk_root :   ['e' 'c' 'b' 'r' '?']
stalk_surface_above_ring :   ['s' 'f' 'k' 'y']
stalk_surface_below_ring :   ['s' 'f' 'y' 'k']
stalk_color_above_ring :   ['w' 'g' 'p' 'n' 'b' 'e' 'o' 'c' 'y']
stalk_color_below_ring :   ['w' 'p' 'g' 'b' 'n' 'e' 'y' 'o' 'c']
veil_type :   ['p']
veil_color :   ['w' 'n' 'o' 'y']
ring_number :   ['o' 't' 'n']
ring_type :   ['p' 'e' 'l' 'f' 'n']
spore_print_color :   ['k' 'n' 'u' 'h' 'w' 'r' 'o' 'y' 'b']
population :   ['s' 'n' 'a' 'v' 'y' 'c']
habitat :   ['u' 'g' 'm' 'd' 'p' 'w' 'l']


In [62]:
#| echo: false
data.drop('veil_type', axis = 1, inplace = True)

We can see that the column `stalk_root` has a non-alphanumeric value and it might need some munging.
According to the dataset's documentation, the value '?' in `stalk_root` means that they are missing or unknown stalk root data. 

Let's see how many of these missing values we have to decide if it'd be okay to drop these rows. 

In [63]:
#| echo: true
vals = data['stalk_root'].value_counts().index.values.tolist()

NA_count = data['stalk_root'].value_counts().values

NA_frac = data['stalk_root'].value_counts().to_list()
NA_frac = [i/sum(NA_frac) for i in NA_frac]

pd.DataFrame(zip(NA_count,NA_frac), columns=['Count','Fraction'], index= vals)

,Count,Fraction
b,3776,0.464796
?,2480,0.305268
e,1120,0.137863
c,556,0.068439
r,192,0.023634


So, now we can see that if we drop the missing values in this column we're losing 30% of our data which accounts for about 2500 instances. 
Dropping the rows is not the best solution in this case. 
Therefore, we'll try to impute using KNN.
Before that, the categorical value must be numerically encoded/labelled from 0 to n. 

Let's see the order of values in this column before label encoding

In [64]:
data.stalk_root.unique().tolist()

['e', 'c', 'b', 'r', '?']

Let's see the corresponding label to '?' values so we can impute them

In [65]:
#| echo: true

le = preprocessing.LabelEncoder()
encoded_data = deepcopy(data)
for i in encoded_data.columns.tolist():
    encoded_data[i]= le.fit_transform(encoded_data[i])

encoded_data['stalk_root'].unique().tolist()

[3, 2, 1, 4, 0]

The corresponding label to '?' is 0. 
But, for the models to impute the missing data, we should replace each 0 with a NaN. 

In [66]:
#| echo: true
print("Stalk root column value counts before imputation: \n", encoded_data.replace({'stalk_root': {0: np.nan}}).stalk_root.value_counts(), '\n')
imputer = KNNImputer(missing_values = np.nan, n_neighbors=5, weights = 'distance')
imputer.fit_transform(encoded_data[['stalk_root']])
print("Stalk root column value counts before imputation: \n", encoded_data.stalk_root.value_counts())

Stalk root column value counts before imputation: 
 1.0    3776
3.0    1120
2.0     556
4.0     192
Name: stalk_root, dtype: int64 

Stalk root column value counts before imputation: 
 1    3776
0    2480
3    1120
2     556
4     192
Name: stalk_root, dtype: int64


We can see that KNNImputer didn't give us any useful results and we're again back on square 1. Therefore, we'll just drop the column.

In [67]:
encoded_data.drop('stalk_root', axis = 1, inplace = True)

Now, let's see if the classes in our dataset are balanced or not.

In [68]:
#| echo: false
vals = data['class'].value_counts().index.values.tolist()

class_count = data['class'].value_counts().values

class_frac = data['class'].value_counts().to_list()
class_frac = [round((i/sum(class_frac))*100, 2) for i in class_frac]

pd.DataFrame(zip(class_count,class_frac), columns=['Count','Fraction'], index= vals)

,Count,Fraction
e,4208,51.8
p,3916,48.2


They are "adequately" balanced and there's no need for any oversampling techniques. 

## EDA 

## Feature & Target Engineering

The only feature and target engineering step we need to perform is label encoding for our categorical variables and the categorical target. Let's identify which class is 1 and which class is 0. 

In [69]:
#| echo: true
print("Class values before encoding: ", data['class'].unique().tolist())
print("Class values after encoding: ", encoded_data['class'].unique().tolist())

Class values before encoding:  ['p', 'e']
Class values after encoding:  [1, 0]


We can see that the `Poisonous (p)` class is now labelled `1`, while the `Edible (e)` class is `0`.

## Baselining

Baseline models are the stepping stone on which AI developers base their initial assumptions of the direction they should take their developing. So, baseline models tend to be rule-based and understandable. Since we're aiming to perform binary classification, we chose to do `LogisticRegression` as a baseline model which we'll use comapre our refined models. 

The second step after label encoding our data to prepare for modeling is to perform standard scaling (z-score).  

In [70]:
#| echo: true
scaled_data = deepcopy(encoded_data)
scaled_data.drop('class', axis=1, inplace = True)

ss = StandardScaler()

ss.fit(scaled_data)
scaled_data = ss.transform(scaled_data)

The third step is to split our data to training and testing sets

75% of the data is used for training while the remaining 25% is for testing

In [71]:
X = scaled_data
y = encoded_data['class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [72]:
# Baseline Logistic Regression Model
lr = LogisticRegression()

# Training 
lr.fit(X_train, y_train)

# Predicting
lr_pred = lr.predict(X_test)

# Evaluating
print('Confusion Matrix :\n', confusion_matrix(y_test, lr_pred))
print()
print('Classification Report :\n', classification_report(y_test, lr_pred))

Confusion Matrix :
 [[965  43]
 [ 56 967]]

Classification Report :
               precision    recall  f1-score   support

           0       0.95      0.96      0.95      1008
           1       0.96      0.95      0.95      1023

    accuracy                           0.95      2031
   macro avg       0.95      0.95      0.95      2031
weighted avg       0.95      0.95      0.95      2031



## Feature Filtering & Dimensionality Reduction

Our dataset has 20 features which is considered high, therefore, means of feature filtering and dimensionality reduction is necessary. For this purpose, we've used 3 feature filtering/selection algorithms (`Variance Thresholding, Chi Square, and LASSO`), as well as `Principal Component Anlaysis` for dimensionality reduction. 

### Variance Thresholding
The first feature filtering method we used was variance thresholding. Features with low variance are considered redundant and non-informative. As a rule of thumb, features that have 5-10% > variance are removed. 

In [73]:
def variance_threshold_selector(data, threshold=0.1):
    selector = VarianceThreshold(threshold)
    selector.fit(data)
    return data.columns[selector.get_support(indices=True)]

variance_threshold_cols = variance_threshold_selector(encoded_data.drop('class', axis = 1))
variance_threshold_cols # Threshold values 5% and 10% are a rule of thumb

Index(['cap_shape', 'cap_surface', 'cap_color', 'bruises', 'odor',
       'gill_spacing', 'gill_size', 'gill_color', 'stalk_shape',
       'stalk_surface_above_ring', 'stalk_surface_below_ring',
       'stalk_color_above_ring', 'stalk_color_below_ring', 'ring_type',
       'spore_print_color', 'population', 'habitat'],
      dtype='object')

We can see that 10% variance resulted in dropping 2 column only (`ring_number` and `veil_color`), while 5% didn't do us any good. 

Let's see the effect of removing these two features on the accuracy using our baseline model.

Actually, the recall and f1-score of the positive class decreased which means that the beaseline model deals better with classifying the positive class than this model. So, this method is not beneficial to us.

### Least Absolute Shrinkage and Selection Operator (LASSO)
The second method is LASSO `L1` which is a regularizaion method that banishes the weights of redundant features to reduce overfitting but they can be used for feature selection. The strength of regularization is determined by the variable `C` which is the strength of the penalty term. As `C` decreases the strength increases.

In [77]:
# Lasso 
# Start by splitting
X = scaled_data
y = encoded_data['class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Select features from the model
sel_ = SelectFromModel(LogisticRegression(C=0.01, penalty='l1', solver='liblinear'))
sel_.fit(X_train, np.ravel(y_train,order='C'))
sel_.get_support()
X_train = pd.DataFrame(X_train)


# To view the set of selected features
selected_feat = X_train.columns[(sel_.get_support())]
print('total features: {}'.format((X_train.shape[1])))
print('selected features: {}'.format(len(selected_feat)))
print('features with coefficients shrank to zero: {}'.format(
np.sum(sel_.estimator_.coef_ == 0)))
print()

sel_col_l1 = list()
[sel_col_l1.append(data.columns[i]) for i in X_train.columns[sel_.get_support(indices=True)].tolist()]
print('Selected columns using Lasso:', '\n',sel_col_l1, '\n')

# Logistic Regression
lr = LogisticRegression(max_iter=500)

# training & prediction
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_test)

# evaluation
print('Confusion Matrix :\n', confusion_matrix(y_test, lr_pred))
print()
print('Classification Report :\n', classification_report(y_test, lr_pred))

total features: 20
selected features: 12
features with coefficients shrank to zero: 8

Selected columns using Lasso: 
 ['cap_shape', 'cap_color', 'bruises', 'gill_attachment', 'gill_spacing', 'gill_size', 'stalk_shape', 'stalk_root', 'stalk_surface_above_ring', 'stalk_surface_below_ring', 'stalk_color_above_ring', 'ring_type'] 

Confusion Matrix :
 [[1006   45]
 [  62  918]]

Classification Report :
               precision    recall  f1-score   support

           0       0.94      0.96      0.95      1051
           1       0.95      0.94      0.94       980

    accuracy                           0.95      2031
   macro avg       0.95      0.95      0.95      2031
weighted avg       0.95      0.95      0.95      2031



LASSO with `C = 0.01` resulted in removing 8 features `{cap_surface, odor, gill_color, stalk_color_below_ring, ring_number, habitat, population, veil_color, spore_print_color}` only to get a similar performance to the beasline model. However, this model is favored since it can get the same performance with 8 less features. 

### Chi Squared
Some shit about it

In [81]:
def select_features(X_train, y_train, X_test):
    fs = SelectKBest(score_func=chi2, k='all')
    fs.fit(X_train, y_train)
    X_train_fs = fs.transform(X_train)
    X_test_fs = fs.transform(X_test)
    return X_train_fs, X_test_fs, fs
X = encoded_data.drop('class', axis = 1)
y = encoded_data['class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

X_train_fs, X_test_fs, fs = select_features(X_train, y_train, X_test)

for i in range(len(fs.scores_)):
    print('Feature %d: %f' % (i, fs.scores_[i]))

Feature 0: 13.923331
Feature 1: 156.656338
Feature 2: 10.472532
Feature 3: 886.750744
Feature 4: 44.583778
Feature 5: 3.023020
Feature 6: 611.221041
Feature 7: 1230.183885
Feature 8: 4419.991956
Feature 9: 28.005064
Feature 10: 169.817457
Feature 11: 158.063962
Feature 12: 98.263204
Feature 13: 85.534474
Feature 14: 4.372503
Feature 15: 17.051667
Feature 16: 1485.578593
Feature 17: 282.895674
Feature 18: 230.428256
Feature 19: 603.775531


#Here we will select features with higher importance score. So by looking at them we may select feature (1, 3, 6, 7, 8, 10, 11, 12, 17, 18, 19 and 20). 
Basically I have selected feature with importance score of more than 100,
considering there is no rule that you have to do selection in this way only,
it is completely on you what importance score you chose as a deciding factor, but one has to take the features with higher score.
#They are:-

#1 - "cap-shape"

#3 - "bruises"

#6 - "gill-spacing"

#7 - "gill-size"

#8 - "gill-color"

#10 - "stalk-root"

#11 - "stalk-surface-above-ring

#12 - "stalk-surface-below-ring"

#17 - "ring-type"

#18 - "spore-print-color"

#19 - "population"

#20 - "habitat"

In [83]:

X = encoded_data[['cap_shape', 'bruises', 'gill_spacing', 'gill_size', 'gill_color', 'stalk_surface_above_ring', 'stalk_surface_below_ring','ring_type', 'spore_print_color','population','habitat']]
y = encoded_data['class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Logistic Regression


lr = LogisticRegression(max_iter=500)

# training & prediction
lr.fit(ss.fit_transform(X_train), y_train)
lr_pred = lr.predict(ss.fit_transform(X_test))

# evaluation
print('Confusion Matrix :\n', confusion_matrix(y_test, lr_pred))
print()
print('Classification Report :\n', classification_report(y_test, lr_pred))

Confusion Matrix :
 [[1006   43]
 [  96  886]]

Classification Report :
               precision    recall  f1-score   support

           0       0.91      0.96      0.94      1049
           1       0.95      0.90      0.93       982

    accuracy                           0.93      2031
   macro avg       0.93      0.93      0.93      2031
weighted avg       0.93      0.93      0.93      2031



This model is better with the negative class which is opposite to the variance thresholding. We'll discard it. 

## Modelling

## Conclusions